# Import Libraries (Required)

Import our essential Libraries


In [ ]:
import os
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten

# Fetch Dataset (Required)

Some shell script to check if the data already exists, if not clone it from git.



In [ ]:
%%shell

if [ ! -d '/tmp/pneumoniaDataset' ]; then 
  mkdir '/tmp/pneumoniaDataset'
fi

if [ ! -d '/tmp/pneumoniaDataset/.git' ]; then 
  git clone "https://github.com/Amzo/xray_images" '/tmp/pneumoniaDataset/'
fi

Cloning into '/tmp/pneumoniaDataset'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 5838 (delta 0), reused 3 (delta 0), pack-reused 5835
Receiving objects: 100% (5838/5838), 1.13 GiB | 37.53 MiB/s, done.
Checking out files: 100% (5857/5857), done.


#Data Setup (Required)

Load our data and generate additional augmented data due to the nature of the small data set. Since the data is already structured into train, test and validate folders, we don't need to split the data here.


In [ ]:
def getData(trainDir, testDir, valDir):
        class_names = os.listdir(trainDir)
        class_types = len(os.listdir(trainDir))

        print('Number of classes for Classification: ',class_types)
        print(f'The class names are {class_names[0]} and {class_names[1]}')
        print('--> Count of Train Images <--')

        for i in class_names:
                print(i + ':' + str(len(os.listdir(trainDir + "/" +i))))
        print('--> Count of Test Images <--')

        for i in class_names:
                print(i + ':' + str(len(os.listdir(testDir + '/' +i))))

        print('--> Count of Validation Images <---')
        for i in class_names:
                print(i + ':' + str(len(os.listdir(valDir + '/' +i))))

        train_datagen = ImageDataGenerator(rescale=1/255.0,
                rotation_range=6,
                zoom_range=0.1,
                horizontal_flip = True,
                vertical_flip = True)

        test_datagen = ImageDataGenerator(rescale=1./255)

        xTrainGen = train_datagen.flow_from_directory(
                trainDir,
                target_size=(224,224),
                shuffle=True,
                batch_size=24,
                class_mode='binary'
        )

        xTestGen = test_datagen.flow_from_directory(
                testDir,
                target_size=(224,224),
                batch_size=16,
                shuffle=True,
                class_mode='binary'
        )

        xValGen = train_datagen.flow_from_directory(
                valDir,
                target_size=(224,224),
                batch_size=32,
                class_mode='binary'
        )

        return xTrainGen, xTestGen, xValGen

Setup the data, the physical_device setup is from running on my own machine to setup GPU training.

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"


inputTrain = '/tmp/pneumoniaDataset/train'
inputTest = '/tmp/pneumoniaDataset/test'
inputValidate = '/tmp/pneumoniaDataset/val'
imageSize = (224,224,3)

xTrain, xTest, xVal = getData(inputTrain, inputTest, inputValidate)


# Data Visualisation

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

In [ ]:
augmentedImages = [xTrain[0][0][0] for i in range(5)]

In [ ]:
plotImages(augmentedImages)

# Model Setup (Required for Training)

Very basic cnn, for setting baseline to test against improvements and other models.

In [ ]:
def myModel():
  print("Defaulting to basic CNN")
  model = Sequential()
  model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(224,224,3)))
  model.add(Conv2D(32, (3, 3), activation='relu'))
  model.add(Conv2D(16, (3, 3), activation='relu'))
  model.add(Flatten())
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='adam',
    loss="binary_crossentropy",
    metrics=["accuracy"])
  
  return model

Return history and the trained model for saving and graphical display

In [ ]:
myCNN = myModel()



Defaulting to basic CNN


In [ ]:
myCNN.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 222, 222, 16)      448       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 220, 220, 32)      4640      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 218, 218, 16)      4624      
_________________________________________________________________
flatten_1 (Flatten)          (None, 760384)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 760385    
Total params: 770,097
Trainable params: 770,097
Non-trainable params: 0
_________________________________________________________________


# Train Model (Optional) - Can Load trained model further down

In [ ]:
def modelTrain(model, xTrain, xVal, batchSize):
  history = model.fit(xTrain,
    epochs=10,
    validation_data=xVal,
    verbose=1,
    batch_size=batchSize
  )

  return history, model

94% train accuracy, 88% validation accuracy and around 85% test accuracy.

In [ ]:
cnnHistory, trainedCNN = modelTrain(myCNN, xTrain, xVal, 32)

Save the model to the mounted drive.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
trainedCNN.save('/content/gdrive/MyDrive/baselineCNN')

In [ ]:
# Store data (serialize)
with open('/content/gdrive/MyDrive/baselineHistory.pickle', 'wb') as outFile:
    pickle.dump(cnnHistory.history, outFile)

# Load Trained Model

Some more shell script to pull the trained model from my one drive. While I can access it if I mount my drive, no one else can. This allows anyone running the notebook to pull the trained model directly from my onedrive with wget.

This function was supposed to be reusable, however, each cell spawns a new shell process, so declared functions and variables aren't reusable in other cells.


In [ ]:
%%shell
function getFile() {
  googleURL="https://docs.google.com/uc?export=download"
  wgetCMD="wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate"
  fileID="$1"
  outFILE="/tmp/$2"

  if [ -f $outFILE ]; then
    rm -f $outFILE
  fi
  
  wget --load-cookies /tmp/cookies.txt "$googleURL&confirm=$($wgetCMD '$googleURL&id=$fileID' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$fileID" -O $outFILE
  rm -rf /tmp/cookies.txt
  
}

# model file ID
getFile "1y0A74bnzR-2cpkMWEhPVRYPYKQrgobv9" "trainedCNN.zip"

outDIR="/tmp/trainedModels"

# check if the directory exits, if not create it
if [ ! -d $outDIR ]; then 
  mkdir -p $outDir; 
fi

# extract the model
unzip -o -d $outDIR $outFILE 

#history json file ID
getFile "10k4nIF7SarOreD6T-bWrhqMQ2EhxJ9I9" "trainedCNNHistory.pickle"


Load the model train history

In [ ]:
with open('/tmp/trainedCNNHistory.pickle', 'rb') as input:
    cnnHistory = pickle.load(input)

Can Finally load the trained model.

In [ ]:
trainedCNN = tf.keras.models.load_model('/tmp/trainedModels/baselineCNN')

# Test Model (Requires a trained or Loaded Model)


In [ ]:
print("Running myCNN model against test set")
ev = trainedCNN.evaluate(xTest)
print("\n%s: %.f%%" % (trainedCNN.metrics_names[1], ev[1]*100))

# Model Visualisation (Requires a trained or loaded model)


Since we can't pickle weakref objects, we pickle the actual history. If the model has been trained, then cnnHistory.history will contain the results, otherwise if we have loaded the pickle file, just cnnHistory will contain the results. To get around this use try, except.

In [ ]:
try:
  plt.plot(cnnHistory.history['accuracy'], label='accuracy')
  plt.plot(cnnHistory.history['val_accuracy'], label = 'val_accuracy')
except AttributeError:
  plt.plot(cnnHistory['accuracy'], label='accuracy')
  plt.plot(cnnHistory['val_accuracy'], label = 'val_accuracy')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = trainedCNN.evaluate(xTest, verbose=2)

In [ ]:
# summarize history for loss
try:
  plt.plot(cnnHistory.history['loss'])
  plt.plot(cnnHistory.history['val_loss'])
except AttributeError:
  plt.plot(cnnHistory['loss'])
  plt.plot(cnnHistory['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()